In [1]:
import pandas as pd

%matplotlib inline 

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

In [2]:
import sklearn

In [3]:
from sklearn.metrics import roc_auc_score, roc_curve

# Overview

Each row represents a home sale.

We try to predict both home sales, and the types of sales.

When `audantic_target`=1, this is the type of transaction we are trying to predict.

```
'audantic_target': 1 or 0, if we are trying to predict

'pid': property id
'did': sales document id
'fips': county id
'zipcode': zipcode
'sfr': bool for is single family home

-- house details
'square_footage': 
'year_built'
'estimated_value'
'length_of_ownership'

-- owner details
'est_household_income_val': estimated income
'mosaic_hh_val': demographic variable for home
'mosaic_zip4_val': demographic variable for neighborhood
'mosaic_diff': difference between home and neighborhood demographics
```

# Import, investigating the data

In [4]:
df = pd.read_csv('df_all_ll.csv.gzip', compression='gzip')

In [5]:
df.head()

,audantic_target,pid,did,fips,zipcode,seller_occupied,square_footage,year_built,estimated_value,length_of_ownership,...,cat_e_e,cat_e,cat_e_r,cat_e_s,cat_g,cat_h,cat_j,cat_m,cat_n,cat_s
0,0,369,603808466,36103,11946.0,0,1260,1997,418000,13.0294,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,383,574522015,36103,11767.0,1,0,0,301000,10.7515,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,960,605013654,36103,11704.0,1,0,0,263000,9.7029,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,973,605452151,36103,11776.0,0,0,0,412000,20.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1136,602447312,36055,14586.0,1,1363,2000,172000,5.0021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df.columns.values

array(['audantic_target', 'pid', 'did', 'fips', 'zipcode',
       'seller_occupied', 'square_footage', 'year_built',
       'estimated_value', 'length_of_ownership',
       'est_household_income_val', 'mosaic_hh_val', 'mosaic_zip4_val',
       'mosaic_diff', 'cat_a', 'cat_ce', 'cat_a_c', 'cat_a_i', 'cat_a_j',
       'cat_a_m', 'cat_a_sk', 'cat_e_b', 'cat_e_e', 'cat_e', 'cat_e_r',
       'cat_e_s', 'cat_g', 'cat_h', 'cat_j', 'cat_m', 'cat_n', 'cat_s'], dtype=object)

In [7]:
y_col = 'audantic_target'

In [8]:
X_cols = [
    'seller_occupied', 'square_footage', 'year_built',
    'estimated_value', 'length_of_ownership',
    'est_household_income_val', 'mosaic_hh_val', 'mosaic_zip4_val',
    'mosaic_diff', 'cat_a', 'cat_ce', 'cat_a_c', 'cat_a_i', 'cat_a_j',
    'cat_a_m', 'cat_a_sk', 'cat_e_b', 'cat_e_e', 'cat_e', 'cat_e_r',
    'cat_e_s', 'cat_g', 'cat_h', 'cat_j', 'cat_m', 'cat_n', 'cat_s']

In [9]:
df[X_cols].describe().round(3)

,seller_occupied,square_footage,year_built,estimated_value,length_of_ownership,est_household_income_val,mosaic_hh_val,mosaic_zip4_val,mosaic_diff,cat_a,...,cat_e_e,cat_e,cat_e_r,cat_e_s,cat_g,cat_h,cat_j,cat_m,cat_n,cat_s
count,1233186.000,1233186.000,1233186.000,1233186.000,1233186.000,1233186.000,1233186.000,1233186.000,1233186.000,1233186.000,...,1233186.000,1233186.000,1233186.000,1233186.000,1233186.000,1233186.000,1233186.000,1233186.000,1233186.000,1233186.000
mean,0.581,1691.112,1827.824,309029.501,11.785,58345.574,41.564,35.860,-5.704,0.034,...,0.001,0.046,0.000,0.000,0.001,0.052,0.001,0.003,0.006,0.005
std,0.493,3085.879,516.699,249186.240,7.964,64498.681,25.046,23.495,18.843,0.181,...,0.032,0.210,0.004,0.012,0.024,0.222,0.031,0.054,0.077,0.069
min,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,-71.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.000,1160.000,1950.000,157000.000,3.822,0.000,19.000,16.000,-8.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,1.000,1557.000,1974.000,244000.000,11.672,40000.000,41.000,31.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
75%,1.000,2119.000,1998.000,383000.000,20.000,87500.000,72.000,59.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
max,1.000,3241353.000,2018.000,3000000.000,66.875,300000.000,72.000,72.000,65.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000


In [10]:
df[['audantic_target']].describe().T.round(3)

,count,mean,std,min,25%,50%,75%,max
audantic_target,1233186.0,0.136,0.343,0.0,0.0,0.0,0.0,1.0


# Model Building

# Model Evaluation